In [9]:
import numpy as np
import pandas as pd
import os
import threading
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator

In [4]:
data_location = '/Users/paramjaswal/Desktop/Dog_Breed/Data/images'

In [5]:
_size = (224, 224)

In [6]:
def load_image(image_path, label, images, labels):
    try:
        _image = load_img(image_path, target_size=_size)
        image_array = img_to_array(_image)
        images.append(image_array)
        labels.append(label)
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")

def load_images(data_loc):
    images = []
    labels = []
    class_names = os.listdir(data_loc)
    class_names.sort()

    threads = []

    for label, class_name in enumerate(class_names):
        class_dir = os.path.join(data_loc, class_name)
        if os.path.isdir(class_dir):
            for image_name in os.listdir(class_dir):
                image_path = os.path.join(class_dir, image_name)
                thread = threading.Thread(target=load_image, args=(image_path, label, images, labels))
                thread.start()
                threads.append(thread)

    for thread in threads:
        thread.join()

    return np.array(images), np.array(labels), class_names

In [7]:
images, labels, class_names = load_images(data_location)
print(f"Loaded {len(images)} images from {len(class_names)} classes.")

Loaded 20580 images from 121 classes.


In [10]:
datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0, 1]
    validation_split=0.2,  # Split 20% of data for validation
    horizontal_flip=True,  # Apply horizontal flip
    zoom_range=0.2,  # Apply zoom
    shear_range=0.2,  # Apply shear transformation
    rotation_range=20,  # Rotate images up to 20 degrees
    brightness_range=[0.8, 1.2]  # Adjust brightness
)

train_generator = datagen.flow_from_directory(
    data_location,
    target_size=(224, 224),  # Resize images to 224x224 pixels
    batch_size=32,
    class_mode='categorical',  # Multi-class classification
    subset='training'  # Use this generator for training
)

val_generator = datagen.flow_from_directory(
    data_location,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Use this generator for validation
)

print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {val_generator.samples}")

Found 16508 images belonging to 120 classes.
Found 4072 images belonging to 120 classes.
Training samples: 16508
Validation samples: 4072
